In [41]:
from dataset import *
from utils import *
import wandb
import torchvision.transforms as transforms
import torch.optim as optim
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor, FasterRCNN
from torchvision.models.detection.backbone_utils import resnet_fpn_backbone
import pandas as pd
import numpy as np

In [85]:
%load_ext autoreload
%autoreload 2
import dataset

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Weights and Biases

In [94]:
wandb.init()

Failed to query for notebook name, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable
wandb: Currently logged in as: kevin-miao (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.10.29 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


## Hyperparameters

In [86]:
### HYPERPARAM

batchsize = 2
in_dim = (300,300)
normalization_data = torch.load('mean-std.pt')
num_classes = 7
print_every = 10

### CONSTANTS
diseases = ['akiec', 'bcc', 'bkl', 'df', 'mel', 'nv', 'vasc']
mapping = {'akiec': 0, 'bcc': 1, 'bkl': 2, 'df': 3, 'mel': 4, 'nv': 5, 'vasc': 6}
inv_mapping = {mapping[k]:k for k in mapping.keys()}


## Data Loading

In [87]:
dataset = SkinData('/', 'final.csv', transform=transforms.Compose([ToTensor, Normalizer(normalization_data)]))
train_data, test_data, valid_data = torch.utils.data.random_split(dataset,[int(0.7 * len(dataset)), int(0.15 * len(dataset)), int(0.15 * len(dataset))+1],  generator=torch.Generator().manual_seed(42))
data_loader_train = torch.utils.data.DataLoader(train_data, batch_size=batchsize, collate_fn = collate_fn)
data_loader_test = torch.utils.data.DataLoader(test_data, batch_size=batchsize, collate_fn = collate_fn)
data_loader_valid =  torch.utils.data.DataLoader(valid_data, batch_size=batchsize, collate_fn = collate_fn)

## Model

In [88]:
def initialize_model():
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes + 1)
    return model

In [89]:
model = initialize_model()

In [90]:
torch.save(model.state_dict(), 'fast_rcnn_v1_state.pt')
torch.save(model, 'fast_rcnn_v1.pt')

In [97]:
for i, d in enumerate(data_loader_train):
    if i == 1:
        break
    im, tar = d
    model.eval()
    print(model(im))


[{'boxes': tensor([[109.0209, 166.9512, 475.0287, 317.7222],
        [128.7600,  93.1953, 479.0657, 253.2561],
        [580.8607, 117.2176, 585.3320, 119.8537],
        [592.4319, 127.0628, 594.7354, 134.3502],
        [579.8682, 116.4488, 585.2988, 118.8200],
        [588.3268, 126.8427, 591.4795, 128.9163],
        [579.7222, 115.1601, 584.8126, 118.0947],
        [ 54.9314, 138.4244, 413.3044, 284.0403],
        [580.4732, 118.5385, 585.6151, 121.0097],
        [593.1764, 126.5080, 595.3911, 134.0532],
        [579.0324, 113.4385, 582.6773, 115.5477],
        [591.2132, 128.9664, 594.5183, 131.2779],
        [577.9556, 114.9260, 583.8983, 117.1593],
        [579.0713, 115.0445, 582.6028, 116.6945],
        [175.8836, 246.4905, 177.0421, 257.4037],
        [580.8677, 116.1735, 587.1180, 119.4146],
        [ 62.1311, 237.0859, 430.6118, 371.2261],
        [174.9132, 245.4762, 176.4623, 256.9381],
        [593.0248, 124.3834, 596.2305, 133.1764],
        [391.3267, 297.3414, 392.1768, 

## Training

In [13]:
# Best Metric
best_mAP = 0
for epoch in range(config.epochs):
    for i, d in enumerate(data_loader_train):

        # Header
        header = 10*"="
        short_header = 5 * '='
        print(header, "Epoch {}".format(epoch), header)

        # Metric Initialization
        metrics = {'val_mAP': ClassMeter(diseases)}
        metrics['train_loss'] = AverageMeter()
        

        # Training
        model.train()

        image, target = d
        image = image.to(device)
        target = target.to(device)
        losses = model(image, target)
        loss = sum(loss for loss in losses.values())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        lr_scheduler.step()

        metrics['train_loss'].update(loss.item(), n=len(target))

        # Log to weights and biases
        wandb.log({'train_loss': metrics['train_loss'].avg})

        # Print every 10 epochs
        if i % (print_every) == 0:
            print('[Epoch {} | {} {}] Training Loss: {}'.format(epoch, i, len(data_loader_train), metrics['train_loss'].avg)

    with torch.no_grad():
        calculate_AP(model, data_loader_valid, metrics)
        
        # Print Summary
        print(short_header, "Validation", short_header)
        current_mAP = metrics['val_mAP'].class_average())
        print('[Val average mAP] : {}'.format(current_mAP)                                                                                                                           
        # Log Values
        # FIXME

        # Log bounding box image sample to WandB
        # FIXME

        # SAVE BEST MODEL
        if current_mAP > best_mAP:
            torch.save(model.state_dict(), wandb.run.dir +
                       '/best.pt'.format(epoch))
            best_mAP = current_mAP

        # SAVE EVERY FIVE EPOCHS
        if epoch % 5 == 0:
            torch.save(model.state_dict(), wandb.run.dir +
                       '/epoch_{}.pt'.format(epoch))

# Test mAP
test_metrics = {'test_mAP': ClassMeter(diseases)}
with torch.no_grad():
    calculate_AP(model, data_loader_test, test_metrics)
print(header, "Test", header)
test_mAP = test_metrics['test_mAP'].class_average())
print('[Test average mAP] : {}'.format(test_mAP) 

# LOG STUFF
# FIXME
        


        

NameError: name 'config' is not defined

In [3]:
import detectron2